In [58]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, Dense, Concatenate, Flatten, Dropout,BatchNormalization
from keras.models import Model, Sequential
import keras
import matplotlib.pyplot as plt
import random
import numpy as np
import tensorflow as tf
# Read the CSV file
data = pd.read_csv("R.csv")

In [59]:

labels = data.copy()

feature_headings = [
    "d1","d2","d3","d4","d5","d6","First Layer","Second Layer"
]

output_values = labels[feature_headings]
input_features = labels[[c for c in labels.columns if c not in feature_headings]]
label_encoder = LabelEncoder()

output_values["First Layer"] = label_encoder.fit_transform(output_values["First Layer"])
output_values["Second Layer"] = label_encoder.transform(output_values["Second Layer"])

unique_materials = pd.unique(data[['First Layer', 'Second Layer']].values.ravel())


num_materials = len(unique_materials)
num_wavelengths = 351

input_train, input_test, output_train, output_test = train_test_split(input_features, output_values, test_size=0.2, random_state=42)
input_train, input_val, output_train, output_val = train_test_split(input_train, output_train, test_size=0.2, random_state=42)

/tmp/ipykernel_132296/1402317043.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_values["First Layer"] = label_encoder.fit_transform(output_values["First Layer"])
/tmp/ipykernel_132296/1402317043.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_values["Second Layer"] = label_encoder.transform(output_values["Second Layer"])


In [129]:
i = Input(num_wavelengths,)

ndl1 = BatchNormalization()(i)
dl1 = Dense(num_wavelengths, activation="PReLU", input_shape=(num_wavelengths,))(ndl1)

m1dl2 = Dense(256, activation="PReLU")(dl1)
m2dl2 = Dense(256, activation="PReLU")(dl1)
tdl2 = Dense(256, activation="PReLU")(ndl1)

concatenated_input = Concatenate()([m1dl2, m2dl2, tdl2])

common_layer = Dense(64)(concatenated_input)

m1dl3 = Dense(256, activation="PReLU")(common_layer)
m2dl3 = Dense(256, activation="PReLU")(common_layer)
tdl3 = Dense(256, activation="PReLU")(common_layer)

out1 = Dense(num_materials, "softmax", name="first_layer")(m1dl3)
out2 = Dense(num_materials, "softmax", name="second_layer")(m2dl3)
thickenesses = Dense(6, "sigmoid", name="thicknesses")(tdl3)

classifier = Model(inputs=i, outputs=[out1,out2,thickenesses])
classifier.compile(
    optimizer="adam",
    loss={
        "first_layer": keras.losses.SparseCategoricalCrossentropy(),
        "second_layer": keras.losses.SparseCategoricalCrossentropy(),
        "thicknesses": "mean_squared_error"
    },
    metrics={
        "first_layer": tf.metrics.SparseCategoricalAccuracy("acc1"),
        "second_layer": tf.metrics.SparseCategoricalAccuracy("acc2"),
        "thicknesses": "accuracy"
    }
)

In [130]:
classifier.fit(
    input_train,
    {
        "first_layer": output_train["First Layer"],
        "second_layer": output_train["Second Layer"],
        "thicknesses": output_train[["d1", "d2", "d3", "d4", "d5", "d6"]]
    },
    epochs=10,
    validation_data=(
        input_val,
        {
            "first_layer": output_val["First Layer"],
            "second_layer": output_val["Second Layer"],
            "thicknesses": output_val[["d1", "d2", "d3", "d4", "d5", "d6"]]
        }
    ),
    batch_size=32
)

Epoch 1/10
600/600 [==============================] - 5s 6ms/step - loss: 1459.1547 - first_layer_loss: 2.9512 - second_layer_loss: 3.2199 - thicknesses_loss: 1452.9834 - first_layer_acc1: 0.1566 - second_layer_acc2: 0.1080 - thicknesses_accuracy: 0.2034 - val_loss: 1440.0433 - val_first_layer_loss: 2.5680 - val_second_layer_loss: 3.0083 - val_thicknesses_loss: 1434.4668 - val_first_layer_acc1: 0.2456 - val_second_layer_acc2: 0.1344 - val_thicknesses_accuracy: 0.1858
Epoch 2/10
600/600 [==============================] - 3s 5ms/step - loss: 1458.5255 - first_layer_loss: 2.6431 - second_layer_loss: 3.0349 - thicknesses_loss: 1452.8473 - first_layer_acc1: 0.2082 - second_layer_acc2: 0.1383 - thicknesses_accuracy: 0.2278 - val_loss: 1439.8402 - val_first_layer_loss: 2.4419 - val_second_layer_loss: 2.9311 - val_thicknesses_loss: 1434.4667 - val_first_layer_acc1: 0.2942 - val_second_layer_acc2: 0.1627 - val_thicknesses_accuracy: 0.2477
Epoch 3/10
600/600 [==============================] - 3s

In [136]:
index = random.randint(0, len(input_test) - 1)
input_sample = input_test.iloc[[index]]
output_sample = output_test.iloc[[index]]
mat_predict = classifier.predict([input_sample])
pred = label_encoder.inverse_transform([np.argmax(mat_predict[0]), np.argmax(mat_predict[1])])
act = label_encoder.inverse_transform([output_sample["First Layer"], output_sample["Second Layer"]])

print(f"modelpredicted: {pred}")
print(f"actual combination: {act}")

print(f"predicted_thicknesses: {mat_predict[2][0]}")
print(f"actual thicknesses: {output_sample.values[0][:6]}")

1/1 [==============================] - 0s 17ms/step
modelpredicted: ['Mn' 'Mg']
actual combination: ['Mg' 'KTaO3']
predicted_thicknesses: [1. 1. 1. 1. 1. 1.]
actual thicknesses: [60 60 30 20 60 40]


/home/clk/repos/l2-transfer-matrix-method/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [137]:
mat_predict

[array([[7.2226249e-02, 4.9381154e-03, 2.4150813e-05, 2.1980939e-02,
         1.3405672e-05, 1.6721595e-02, 1.1273656e-05, 7.4959712e-06,
         1.2992452e-02, 2.2793314e-04, 1.8634672e-01, 1.4656462e-02,
         5.4424326e-03, 2.2721305e-01, 2.2199424e-01, 6.9313813e-03,
         6.0199723e-03, 2.6527300e-06, 1.6992679e-02, 1.2730055e-02,
         3.1961221e-02, 1.6137188e-02, 2.3986079e-02, 8.3818544e-05,
         5.7677142e-03, 3.4168903e-02, 1.6079513e-04, 1.9238483e-03,
         7.0986394e-03, 4.5850862e-02, 1.4524620e-03, 3.9352193e-03]],
       dtype=float32),
 array([[0.05476758, 0.02387645, 0.01805017, 0.02706322, 0.02891464,
         0.03094666, 0.02684937, 0.01894384, 0.03261182, 0.02300763,
         0.07156663, 0.03360134, 0.02771512, 0.07059391, 0.0704021 ,
         0.02462649, 0.02410819, 0.0257853 , 0.03462988, 0.02816422,
         0.02745189, 0.02518246, 0.0295989 , 0.02630394, 0.02285752,
         0.02779222, 0.02063611, 0.01990551, 0.02404341, 0.03585097,
         